## Loading and Preparing Data 

In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy 
from matplotlib import pyplot as plt
import warnings

warnings.filterwarnings('ignore')

# Trainng Samples loaded from Github Repo
ratings_path = 'https://raw.githubusercontent.com/usef-kh/EC503Project/master/Datasets/ml-1m/ratings.dat'
titles_path = 'https://raw.githubusercontent.com/usef-kh/EC503Project/master/Datasets/ml-1m/movies.dat'

ratings = pd.read_csv(ratings_path, sep  = "::", names = ['UserID', 'MovieID', 'Rating', 'Timestamp'])
titles = pd.read_csv(titles_path, sep = "::", names = ['MovieID', 'Title', 'Genres'], encoding ='latin-1')
data = pd.merge(ratings, titles, on='MovieID')

ntotal = data.shape[0]
ntrain = int(np.floor(ntotal*0.85))
ntest = ntotal - ntrain

print(f"ntotal: {ntotal}\nntrain: {ntrain}\nntest: {ntest}")

data.head()

ntotal: 1000209
ntrain: 850177
ntest: 150032


,UserID,MovieID,Rating,Timestamp,Title,Genres
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


### Suffling dataset and splitting into test, train, and valiation

In [2]:
np.random.seed(42)
shuffled_data = data.reindex(np.random.permutation(data.index))

train = deepcopy(shuffled_data)
val = deepcopy(shuffled_data)

train['Rating'][ntrain:] = -1
val['Rating'][:ntrain] = -1

train_mtx = train.pivot_table(index='UserID', columns='Title', values='Rating')
test_mtx = val.pivot_table(index='UserID', columns='Title', values='Rating')

train_mtx = train_mtx.values
test_mtx = test_mtx.values


train_mtx[train_mtx == -1] = np.nan
test_mtx[test_mtx == -1] = np.nan

train_select = ~(np.isnan(train_mtx))
test_select = ~(np.isnan(test_mtx))

movie_means = np.round(2 * np.nanmean(train_mtx, axis = 1))/2
inds = np.where(np.isnan(train_mtx))
train_mtx[inds] = np.take(movie_means, inds[1])

print(np.shape(test_mtx), np.shape(train_mtx))
print(np.shape(test_select), np.shape(train_select))

(6040, 3706) (6040, 3706)
(6040, 3706) (6040, 3706)


#### Obtaining MAE on test set after filling movie means only to compare with performance of SVD 

In [3]:
from sklearn.metrics import mean_absolute_error as mae

result = train_mtx[test_select].flatten()
ground_truth = test_mtx[test_select].flatten()

print("MAE by only filling mean value:")
print(mae(result,ground_truth))

MAE by only filling mean value:
0.9494841100565212


## Implementing Matrix Factorization using SVD

### Tuning hyperparameters using hold out validation
#### SVD essentially performs dimention reduction through selecting the rank of the resultant matricies, specifically Sigma.

#### On a high level of understanding:
#### - U corresponds to the users' features
#### - V' captures movies' features. 

#### Obviously, the higher the rank, the more information is perserved during the reduction. However, the larger its value, the less we are learning the actual data and the more we are overfitting the training set. For this, we will be corss validating our results and minimizing the mse over a validation set to tune the value of k.

In [0]:
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_absolute_error as mae

tmax = 100
MAE = []
reconstruction = deepcopy(train_mtx)
for t in range(tmax):
  U, sigma, Vt = svds(reconstruction, 50)
  sigma = np.diag(sigma)

  reconstruction = np.round(2*(np.dot(np.dot(U, sigma), Vt)))/2
  reconstruction[train_select] = train_mtx[train_select]

  reconstruction[reconstruction > 5] = 5
  reconstruction[reconstruction < 0.5] = 0.5
  
  result = reconstruction[test_select].flatten() 
  ground_truth = test_mtx[test_select].flatten()

  MAE.append(mae(result,ground_truth))

In [0]:
plt.plot(list(range(tmax)), MAE)
plt.xlabel('Number of Iterations')
plt.ylabel('Test MAE')
plt.xticks(np.arange(1, tmax, 2))
plt.xlim(0,tmax)
plt.title("MAE vs Number of Iterations")
plt.grid()